# Example of a simple setup of a few mPMTs to determine timing offsets 

 * This is intended to be done early in the testing of mPMTs: a valuable consistency check



In [1]:
import k3d
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, "..")

from TimingCalib.WCD_calibrator import WCD_calibrator
from TimingCalib.WCD_multilaterator import WCD_multilaterator
from TimingCalib.WCD import WCD
from TimingCalib.MPMT import MPMT
from TimingCalib.PMT import PMT
from TimingCalib.LED import LED

In [2]:
# create a new setup with some mPMTs and optionally a beacon
# - wcd_kind: name of the setup
# - mpmt: list of mpmt dictionaries
# - beacon_kind: name of the beacon device
# - beacon: beacon dictionary
# - beacon_delay_sig: std dev of beacon delay

def define_simple_setup(wcd_kind, mpmts, beacon_kind="", beacon=None, beacon_delay_sig = 1.):
    WCD.prop_mean[wcd_kind] = WCD.def_prop_mean.copy()
    WCD.prop_scale[wcd_kind] = WCD.def_prop_scale.copy()
    WCD.prop_var[wcd_kind] = WCD.def_prop_var.copy()
    
    WCD.mpmts_design[wcd_kind] = mpmts
    
    if beacon_kind != "":
        LED.prop_mean[beacon_kind] = LED.def_prop_mean.copy()
        LED.prop_scale[beacon_kind] = LED.def_prop_scale.copy()
        LED.prop_var[beacon_kind] = LED.def_prop_var.copy()

        LED.prop_scale[beacon_kind]['delay'] = delay_sig
        
        WCD.calibs_design[wcd_kind] = [beacon]

In [3]:
# setup 3 mPMTs in a room: floor is x-y plane.

def get_wcd(led_cone_angle):

    wcd_kind = 'test_3'
    mpmt_kind = 'M2' # dome style mPMT

    radius = 2000. # mm radius of circle of mPMT centres
    phi = np.pi/6 # phi angle of first mPMT
    loc_sig = [2., 2., 2.] # mm std dev
    rot_axes = 'yz' # rotation axes of mPMT
    rot_sig = [0.01, 0.01] # rad std dev

    mpmts = []
    for i in range(3):
        location = [radius*np.cos(phi), radius*np.sin(phi), 0.]
        rot_angles = [np.pi/2., phi - np.pi]
        mpmts.append({
            'kind': mpmt_kind,
            'loc': location,
            'loc_sig': loc_sig,
            'rot_axes': rot_axes,
            'rot_angles': rot_angles,
            'rot_angles_sig':rot_sig})    
        phi += 2.*np.pi/3.

    # define the setup of the mpmts
    define_simple_setup(wcd_kind, mpmts)

    # set the cone angle of the LEDs (default is 1 rad)
    LED.prop_mean['L1']['cone_angle'] = led_cone_angle

    # create the detector: creates true orientations and properties of all devices
    wcd = WCD(None, wcd_kind, {}, {})

    # remove all but first of the LEDs
    for mpmt in wcd.mpmts:
        led = mpmt.leds[0]
        mpmt.leds = [led]

    return wcd

In [4]:
# Show all mPMTs, PMTs, LEDs in the setup

wcd = get_wcd(1.0)

plot = k3d.plot()

# mpmt coordinates
color_z = 0x0000ff
color_x = 0xff0000
origins = []
z_vecs = []
x_vecs = []
vec_length = 100. # length of x,z vectors to show for each pmt

color_mpmt = 0xabb2b9
n_point_mpmt = 8
indices_mpmt = []
for i in range(n_point_mpmt):
    indices_mpmt.append([i,(i+1)%n_point_mpmt])
    
n_point_ft = 20
indices_ft = []
for i in range(n_point_ft):
    indices_ft.append([i,(i+1)%n_point_ft])
    
# pmt faces:
color_pmt = 0xdc7633
n_point_pmt = 20
indices_pmt = []
for i in range(n_point_pmt):
    indices_pmt.append([i,(i+1)%n_point_pmt])
    
# led coordinates
led_origins = []
led_z_vecs = []
led_vec_length = 30. # length of z vectors to show for each led

color_led = 0x00ff00
    
# draw the extent of the mpmt baseplates
for i_mpmt,mpmt in enumerate(wcd.mpmts):
    
    location, direction_x, direction_z = mpmt.get_orientation('design')
    # lists to show mPMT coordinate systems
    z_vec = list(np.array(direction_z)*vec_length)
    x_vec = list(np.array(direction_x)*vec_length)
    origins.append(location)
    z_vecs.append(z_vec)
    x_vecs.append(x_vec)
    
    baseplate_points = np.array(mpmt.get_xy_points('design'),dtype=np.float32)
    plt_baseplate = k3d.lines(baseplate_points, indices_mpmt, indices_type='segment', color=color_mpmt)
    plot += plt_baseplate
    
    feedthrough_points = np.array(mpmt.get_xy_points('design', feature='feedthrough'),dtype=np.float32)
    plt_feedthrough = k3d.lines(feedthrough_points, indices_ft, indices_type='segment', color=color_mpmt)
    plot += plt_feedthrough

    # k3d complains about the following not being float32!
    plt_text = k3d.text(str(i_mpmt), position=location, reference_point='cc', size=1., label_box=False, color=color_mpmt)
    plot += plt_text
    
    for i_pmt, pmt in enumerate(mpmt.pmts):

        # k3d expects ndarray of float32 for segments
        circle_points = np.array(pmt.get_circle_points(n_point_pmt,'design'),dtype=np.float32)
        plt_circle = k3d.lines(circle_points, indices_pmt, indices_type='segment', color=color_pmt)
        plot += plt_circle

    for i_led, led in enumerate(mpmt.leds):
        location, direction_x, direction_z = led.get_orientation('design')
        z_vec = list(np.array(direction_z)*led_vec_length)
        led_origins.append(location)
        led_z_vecs.append(z_vec)
    
    
#plt_z_vecs = k3d.vectors(origins=origins, vectors=z_vecs, color=color_z, head_size=250.)
#plot += plt_z_vecs

#plot led locations and axis
led_locations = np.array(led_origins, dtype=np.float32)    
plt_leds = k3d.points(positions=led_locations,
                        point_size=8.,
                        shader='3d',
                        color=color_led)
plot += plt_leds

plt_led_z_vecs = k3d.vectors(origins=led_origins, vectors=led_z_vecs, color=color_z, head_size=50.)
plot += plt_led_z_vecs

plot.display()

Output()

In [5]:
def calibrate(wcd, print_results):
    """ Run calibration and optionally print results"""
    my_wcd = wcd
    my_wcd_calibrator = WCD_calibrator(my_wcd)
    chi2, n_dof, pars, mean_sigma_t, n_term_t = my_wcd_calibrator.calibrate()

    if print_results:
        n_mpmt = len(my_wcd.mpmts)
        m_pmt = len(my_wcd.mpmts[0].pmts)
        frac_signal = 1. * n_term_t / (n_mpmt * n_mpmt * m_pmt)
        print('chi2 = {0:.1f}, n_dof = {1:d}, n_pars = {2:d}, '.\
              format(chi2, n_dof, len(pars)) + \
              'mean_s_t = {0:.3f}, frac_signal = {1:.3f}, n_t = {2:d}'.\
              format(mean_sigma_t, frac_signal, n_term_t))
        
        print('MPMT delays:')
        mpmt = my_wcd.mpmts[0]
        print('prior mean = {0:.2f}  prior half-range = {1:.2f}' \
              .format(MPMT.prop_mean[mpmt.kind]['clock_offset'], \
                      MPMT.prop_scale[mpmt.kind]['clock_offset']))
        diff = []
        for i in range(1,n_mpmt):
            diff.append(pars[n_mpmt+i-1] - my_wcd.mpmts[i].prop_true['clock_offset'])
        print('Residuals mean = {0:.2f}  Residual sig = {1:.2f}' \
              .format(np.mean(diff), np.std(diff)))
        
        print('LED delays:')
        led = my_wcd.mpmts[0].leds[0]
        print('prior mean = {0:.2f}  prior sig = {1:.2f}' \
              .format(LED.prop_mean[led.kind]['delay'], LED.prop_scale[led.kind]['delay']))
        diff = []
        for i in range(n_mpmt):
            diff.append(pars[i] - my_wcd.mpmts[i].leds[0].prop_true['delay'])
        print('Residuals mean = {0:.2f}  Residual sig = {1:.2f}' \
              .format(np.mean(diff), np.std(diff)))
            
        print('PMT delays:')
        pmt = my_wcd.mpmts[0].pmts[0]
        print('prior mean = {0:.2f}  prior sig = {1:.2f}' \
              .format(PMT.prop_mean[pmt.kind]['delay'], PMT.prop_scale[pmt.kind]['delay']))
        diff = []
        for j in range(n_mpmt):
            for k in range(m_pmt):
                p = j*m_pmt + k
                diff.append(pars[n_mpmt*2-1+p] - my_wcd.mpmts[j].pmts[k].prop_true['delay'])
        print('Residuals mean = {0:.2f}  Residual sig = {1:.2f}' \
              .format(np.mean(diff), np.std(diff)))
            
    return chi2, n_dof, pars, mean_sigma_t, n_term_t

In [7]:
wcd = get_wcd(1.0) # cone angle 1.0 rad = 57 degrees
chi2, n_dof, pars, mean_sigma_t, n_term_t = calibrate(wcd, True)

chi2 = 96.8, n_dof = 91, n_pars = 62, mean_s_t = 0.081, frac_signal = 0.544, n_t = 93
MPMT delays:
prior mean = 0.00  prior half-range = 100.00
Residuals mean = 0.33  Residual sig = 0.15
LED delays:
prior mean = 4.00  prior sig = 1.00
Residuals mean = -0.07  Residual sig = 0.22
PMT delays:
prior mean = 4.00  prior sig = 0.50
Residuals mean = 0.08  Residual sig = 0.19


In [6]:
wcd = get_wcd(0.6) # cone angle 0.6 rad = 34 degrees
chi2, n_dof, pars, mean_sigma_t, n_term_t = calibrate(wcd, True)  

chi2 = 26.2, n_dof = 43, n_pars = 62, mean_s_t = 0.079, frac_signal = 0.263, n_t = 45
MPMT delays:
prior mean = 0.00  prior half-range = 100.00
Residuals mean = -0.12  Residual sig = 0.56
LED delays:
prior mean = 4.00  prior sig = 1.00
Residuals mean = 0.05  Residual sig = 0.86
PMT delays:
prior mean = 4.00  prior sig = 0.50
Residuals mean = -0.05  Residual sig = 0.29
